In [5]:
import random
import itertools
import numpy as np
import pandas as pd
from collections import Counter
from generate_pick_rules import generate_pick_rules

In [3]:
def count_picks(df):
    result = pd.DataFrame(columns=["champions", "picks_count", "positions"])

    positions = ["top", "jungle", "mid", "bottom", "support"]
    for position in positions:
        temp = pd.DataFrame(
            Counter(df[position]).items(),
            columns=["champions", "picks_count"]
        )
        temp["positions"] = position
        result = pd.concat([result, temp]).reset_index(drop=True)

    return result

In [6]:
# You can get a "match_picks.csv" by running the "get_picks_info.py"
df_count = count_picks(pd.read_csv("match_picks.csv"))
df_count.head()

,champions,picks_count,positions
0,Lissandra,8,top
1,Poppy,66,top
2,Sylas,63,top
3,Jax,48,top
4,Vladimir,197,top


In [7]:
df_rules = generate_pick_rules()
df_rules.head()

,picked,recommend,confidence,lift
0,"(Maokai,)",Nocturne,1.0,32.117647
1,"(Elise, Rengar)",Twisted Fate,1.0,84.000000
2,"(Elise, Twisted Fate)",Rengar,1.0,819.000000
3,"(Rengar, Twisted Fate)",Elise,1.0,20.475000
4,"(Talon,)",Sylas,0.5,4.321900


In [11]:
def generate_first_pick(df):
    positions = ["top", "jungle", "mid", "bottom", "support"]
    first_pick_position = random.choice(positions)

    df = df.loc[df["positions"] == first_pick_position, :]
    df = df.sort_values(by="picks_count", ascending=False).reset_index(drop=True)
    n = np.random.randint(0, 2)

    return {first_pick_position: df.iloc[n, 0]}

In [12]:
def add_random_pick(df, picks):
    positions = ["top", "jungle", "mid", "bottom", "support"]
    positions = list(set(positions) - set(picks.keys()))
    champions = df[df["positions"].isin(positions)]
    champions = champions[champions["picks_count"] > 10].values

    while True:
        next_pick = random.choice(champions)
        next_pick_champion = next_pick[0]
        next_pick_position = next_pick[2]
        if next_pick_champion not in picks.values():
            break
    picks[next_pick_position] = next_pick_champion

    return picks

In [13]:
def add_next_pick(df_count, df_rules, picks):
    for i, _ in enumerate(list(picks.values()), 1):
        for picked in itertools.permutations(list(picks.values()), len(picks) - i + 1):
            rules = df_rules.loc[df_rules["picked"] == str(picked), :] \
                            .sort_values(by="lift", ascending=False) \
                            .reset_index(drop=True)
            if len(rules) > 0:
                for i, next_pick in enumerate(rules["recommend"]):
                    if next_pick in picks.values():
                        continue
                    possible_positions = df_count[
                        df_count["champions"] == next_pick
                    ].sort_values(by="picks_count", ascending=False)[
                        "positions"
                    ].values
                    for position in possible_positions:
                        if position not in picks.keys():
                            picks[position] = next_pick
                            return picks

    return add_random_pick(df_count, picks)

In [18]:
def generate_virtual_picks(df_count, df_rules, n=5):
    df = pd.DataFrame(
        columns=["top", "jungle", "mid", "bottom", "support"]
    )
    for i in range(n):
        picks = generate_first_pick(df_count)
        while (len(picks) < 5):
            picks = add_next_pick(df_count, df_rules, picks)
        temp = pd.DataFrame([picks])
        temp = temp[["top", "jungle", "mid", "bottom", "support"]]
        df = pd.concat([df, temp]).reset_index(drop=True)
    return df

In [19]:
df = generate_virtual_picks(df_count, df_rules, 10)
df

,top,jungle,mid,bottom,support
0,Rumble,Jarvan IV,Zoe,Lucian,Bard
1,Kayle,Jarvan IV,Lissandra,Caitlyn,Tahm Kench
2,Aatrox,Kindred,Lissandra,Xayah,Taric
3,Jarvan IV,Rek'Sai,Renekton,Ezreal,Fiddlesticks
4,Riven,Lee Sin,Twisted Fate,Ezreal,Pyke
5,Vladimir,Sylas,Corki,Jinx,Galio
6,Kennen,Skarner,Lissandra,Viktor,Pyke
7,Kennen,Camille,Neeko,Xayah,Braum
8,Shen,Camille,Tristana,Kog'Maw,Braum
9,Poppy,Xin Zhao,Corki,Tristana,Alistar
